# Jupyter notebook to compute metrics for performance comparison
## importing packages and building dataset path

In [3]:
from pathlib import Path
import lpips
import skimage.metrics as metrics

current_path = Path('.').resolve()
print(str(current_path))

png_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Source: ",png_path)

/home/filippo/DataspellProjects/ImageCompressionAI
Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/PNG_IMAGES
